In [ ]:
!pip install --upgrade pip
!pip install tweepy
!pip install json
!pip install googletrans
!pip install transformers
!pip install sentencepiece
import transformers
from transformers import pipeline
import tweepy
import json
import pandas as pd

consumer_key = "uz2DmvQ65q4G1El8bLUVdHUh0"
consumer_secret = "Empr5UjFWyWivwPHT4krnhKACzJ8tIPdck2ydcmteV2Lq8zLOK"
access_token = "1573341649937338371-feh5V4QMc3UkWd21beCeb0KZIwFzCr"
access_token_secret = "04wbkK2UNpZ964mDRk3RHkHeSvGmbJAXX2wbhEeWgB5gX"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
num_tweets=10000 #Cantidad


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def extr(user):
  dic1=pd.DataFrame() #Dataframe que almacenará los tweets
  i=1
  ncat=200
  for tweet in tweepy.Cursor(api.user_timeline, count=500,screen_name = str(user),
                            exclude_replies = True, include_rts = True,
                            tweet_mode="extended").items(num_tweets):
    dic1=dic1.append(pd.json_normalize(tweet._json))
    i=i+1
    if i%ncat==0:
      print(str(i)+" tweets extraidos")
  df=pd.DataFrame(dic1)
  (pd.DataFrame(df['full_text'])).to_excel(str(user)+'.xlsx') 

In [ ]:
extr('Nequi') #ES
extr('DaviPlata') #ES
extr('nubank') #POR
extr('Chime') #EN
extr('uala_arg') #ES
extr('C6Bank') #POR
extr('Wise') #EN

### Nota:
Los archivos fueron traducidos fuera del notebook para asegurar mayor precisión en la traducción

In [ ]:
Nequi=pd.read_excel("NequiEN.xlsx",header=0,sheet_name=0)
DaviPlata=pd.read_excel("DaviPlataEN.xlsx",header=0,sheet_name=0)
Uala=pd.read_excel("UalaEN.xlsx",header=0,sheet_name=0)
C6Bank=pd.read_excel("C6BankEN.xlsx",header=0,sheet_name=0)
Nubank=pd.read_excel("NubankEN.xlsx",header=0,sheet_name=0)
Wise=pd.read_excel("Wise.xlsx",header=0,sheet_name=0)
Chime=pd.read_excel("Chime.xlsx",header=0,sheet_name=0)

In [ ]:
DF=pd.concat([Nequi,DaviPlata,Uala,C6Bank,Nubank,Wise,Chime], axis=0)

In [50]:
DF["full_text2"] = (DF["full_text"]
                               .str.lower()
                               .str.replace("nequi","Neobank ", regex=True)
                               .str.replace("daviplata","Neobank ", regex=True)
                               .str.replace("uala","Neobank ", regex=True)
                               .str.replace("wise","Neobank ", regex=True)
                               .str.replace("c6bank","Neobank ", regex=True)
                               .str.replace("nubank","Neobank ", regex=True)
                               .str.replace("chime","Neobank ", regex=True)
                               .str.strip()
                              ) 
DF[["full_text","full_text2"]].tail(10)
DF["full_text"] = DF["full_text2"]
DF = DF.drop(columns="full_text2")

In [54]:
DF1=pd.DataFrame((DF["full_text"]+".").str.capitalize())

In [56]:
with pd.ExcelWriter("TuitsNeobancos.xlsx") as writer:
    DF1.to_excel(writer, sheet_name="Tuits", index=False)

## Opción 2 para extraer los TimeLine


In [ ]:
# Iniciamos un array que va a contener los datos
simple_list = []
# Iniciamos el for con Cursor
# Usamos algunos parámetros extra para excluir respuestas, retweets y traer el texto completo de cada tweet
for status in tweepy.Cursor(api.user_timeline, count=500,screen_name = "@nubank",
                            exclude_replies = True, include_rts = True,
                            tweet_mode="extended").items(500):
 # Agregamos el texto, fecha, likes, retweets y hashtags al array
    simple_list.append([status.full_text, status.created_at, status.favorite_count,
                        status.retweet_count, [h["text"] for h in status.entities["hashtags"]],[h["screen_name"] for h in status.entities["user_mentions"]]])
# Convertimos el array en un DataFrame y nombramos las columnas
simple_list = pd.DataFrame(simple_list, columns=["Text", "Created at", "Likes", "Retweets", "Hashtags", "Mentions"])
# Guardamos en el directorio en que estamos trabajando
simple_list.to_excel('nubank.xlsx', index = None, header=True)